# Deep-Live-Cam on Google Colab
Real-time face swap with Gradio interface

In [ ]:
# Install system dependencies
!apt update -qq
!apt install -y ffmpeg

# Clone repository
!git clone https://github.com/Mayank-kanojiya/liveswap.git
%cd liveswap

In [ ]:
# Install Python dependencies
!pip install -q gradio opencv-python insightface onnxruntime-gpu torch torchvision
!pip install -q git+https://github.com/xinntao/BasicSR.git@master
!pip install -q git+https://github.com/TencentARC/GFPGAN.git@master

In [ ]:
# Download required models
import os
os.makedirs('models', exist_ok=True)

!wget -q -O models/GFPGANv1.4.pth https://huggingface.co/hacksider/deep-live-cam/resolve/main/GFPGANv1.4.pth
!wget -q -O models/inswapper_128_fp16.onnx https://huggingface.co/hacksider/deep-live-cam/resolve/main/inswapper_128_fp16.onnx

In [ ]:
# Create Gradio interface
import gradio as gr
import cv2
import tempfile
import sys
sys.path.append('.')

from modules.processors.frame.face_swapper import process_frame
from modules.face_analyser import get_one_face
import modules.globals

def swap_face(source_image, target_file):
    if source_image is None or target_file is None:
        return None, "Please provide both source image and target media"
    
    try:
        modules.globals.source_path = source_image
        
        # Check if target is image or video
        if target_file.name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            # Image processing
            result_path = tempfile.mktemp(suffix='.jpg')
            frame = cv2.imread(target_file.name)
            source_face = get_one_face(cv2.imread(source_image))
            result_frame = process_frame(source_face, frame)
            cv2.imwrite(result_path, result_frame)
            return result_path, "Face swap completed!"
            
        else:
            # Video processing
            result_path = tempfile.mktemp(suffix='.mp4')
            cap = cv2.VideoCapture(target_file.name)
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(result_path, fourcc, fps, (width, height))
            
            source_face = get_one_face(cv2.imread(source_image))
            
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                result_frame = process_frame(source_face, frame)
                out.write(result_frame)
            
            cap.release()
            out.release()
            return result_path, "Video face swap completed!"
            
    except Exception as e:
        return None, f"Error: {str(e)}"

# Create interface
with gr.Blocks(title="Deep-Live-Cam") as demo:
    gr.Markdown("# Deep-Live-Cam - Face Swap")
    
    with gr.Row():
        with gr.Column():
            source_image = gr.Image(label="Source Face Image", type="filepath")
            target_media = gr.File(label="Target Image/Video")
            swap_btn = gr.Button("Swap Face", variant="primary")
            
        with gr.Column():
            result_media = gr.File(label="Result")
            status_text = gr.Textbox(label="Status", interactive=False)
    
    swap_btn.click(
        fn=swap_face,
        inputs=[source_image, target_media],
        outputs=[result_media, status_text]
    )

demo.launch(share=True, debug=True)